### Opening

In [1]:
import pandas as pd

In [ ]:
import os
import pandas as pd

folder = "/Users/luisfaria/Desktop/sEngineer/masters_SWEAI/2025-T2/T2-REM/assignments/Assessment2/sales"

# If in codespaces
# folder = "/workspaces/masters-swe-ai/2025-T2/T2-REM/assignments/Assessment2/sales"

# List to hold dataframes
dataframes = []

# Read all Excel files and append to list
for file in os.listdir(folder):
    if file.endswith(".xlsx"):
        filepath = os.path.join(folder, file)
        df = pd.read_excel(filepath)
        dataframes.append(df)

# Concatenate all dataframes into one
if dataframes:
    combined_df = pd.concat(dataframes, ignore_index=True)
    print("Data combined successfully.")
else:
    print("No Excel files found.")

# Optional: print the first few rows
combined_df.sample(5)

### Treating

In [3]:
df.columns

Index(['ID orçamento', 'Data orçamento', 'Status', 'Revenda', 'Data venda',
       'Mês venda', 'Hora venda', 'Unidade', 'Consultor', 'Avaliador',
       'Valor tabela', 'Valor desconto', 'Valor líquido', 'Grupo procedimento',
       'Procedimento', 'Quantidade', 'Valor tabela item',
       'Valor % desconto item', 'valor desconto item', 'Valor liquido item',
       'Fonte do cadastro do cliente', 'Nome cliente', 'CPF cliente',
       'ID cliente', 'Email do cliente', 'Telefone(s) do cliente',
       'Data nascimento cliente', 'Profissão cliente', 'Cortesia?'],
      dtype='object')

In [4]:
desired_columns = [
        'ID orçamento', 'Status','Data venda', 'Mês venda',
        'Unidade', 'Valor líquido', 'ID cliente']

df_filtered = combined_df[desired_columns]

# treating
df_filtered = df_filtered.loc[df_filtered['Status'] == 'Finalizado']
df_filtered['Valor líquido'] = df_filtered['Valor líquido'].fillna(0)
df_filtered.sort_values(by='Valor líquido', ascending=False)

# total sum of 'Valor líquido'
df_filtered['Valor líquido'].sum()



17719676.38

In [5]:
df_filtered['Data venda'] = pd.to_datetime(df_filtered['Data venda'])
df_filtered['Year'] = df_filtered['Data venda'].dt.year
df_filtered['Month'] = df_filtered['Data venda'].dt.month

### Checking

In [6]:
# convert 'Valor líquido' to float
df_filtered['Valor líquido'] = df_filtered['Valor líquido'].astype(float)


df_filtered.groupby(['Unidade', 'Month', 'Year'])['Valor líquido'].sum().unstack(fill_value=0)

Year                       2024      2025
Unidade        Month                     
ALPHAVILLE     1           0.00  76386.00
               2           0.00  82787.04
               3           0.00  60769.91
               12      92290.08      0.00
BELO HORIZONTE 1      111643.77      0.00
...                         ...       ...
TUCURUVI       12     152590.93      0.00
VILA MASCOTE   1           0.00  94511.00
               2           0.00  60550.99
               3           0.00  50164.01
               12      84607.97      0.00

[88 rows x 2 columns]

In [7]:
df_tucuruvi = df_filtered.loc[df_filtered['Unidade'] == 'TUCURUVI']
df_tucuruvi.groupby(['Unidade', 'Month', 'Year'])['Valor líquido'].sum().unstack(fill_value=0)

Year                 2024       2025
Unidade  Month                      
TUCURUVI 1      187160.08  164754.90
         2           0.00  155763.51
         3           0.00  160833.49
         12     152590.93       0.00

### Preparing to merge

In [8]:
df_filtered.columns

Index(['ID orçamento', 'Status', 'Data venda', 'Mês venda', 'Unidade',
       'Valor líquido', 'ID cliente', 'Year', 'Month'],
      dtype='object')

In [9]:
df_filtered['month'] = df_filtered['Year'].astype(str) + '-' + df_filtered['Month'].astype(str).str.zfill(2)

In [10]:
df_sales_summary = df_filtered.groupby(['month', 'Unidade']).agg(
    total_valor_liquido=('Valor líquido', 'sum'),
    count_of_sales=('ID orçamento', 'count')
).reset_index()

# Optional: Rename 'Unidade' to 'Store' to match NPS dataframe column name
df_sales_summary = df_sales_summary.rename(columns={'Unidade': 'Store'})

In [13]:
df_sales_summary

# Save the cleaned data
df_sales_summary.to_csv('revenue_data.csv', index=False)
